In [1]:
import pickle
import tools.task3_scorer_onefile
from utils import *
import numpy as np
import logging

import pandas as pd
import torch
from pytorch_pretrained_bert import (BasicTokenizer, BertConfig,
                                     BertForTokenClassification, BertTokenizer)
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as f1
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler, TensorDataset
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
import os 
import itertools

import matplotlib.pyplot as plt 

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
def make_logger() -> None:
    if not os.path.exists("./exp/{}/{}".format(opt["classType"], opt["expID"])):
            try:
                os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))
            except FileNotFoundError:
                os.mkdir("./exp/{}".format(opt["classType"]))
                os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))
    
    logging.basicConfig(
    filename= ("./exp/{}/{}/log.txt".format(opt["classType"], opt["expID"])),
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s, %(message)s')

    logging.getLogger().addHandler(logging.StreamHandler())

In [2]:

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(
            model.state_dict(), './exp/{}/{}/best_model.pth'.format(opt["classType"], opt["expID"]))
        #torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


## Preprocess

In [3]:
import pandas as pd
import pickle
import ast
from utils import *


def pad_sequences(sequences: list, batch_first: bool = True, padding_value: int = 0, max_len: int = 0):
    tmp = torch.Tensor(sequences[0])
    max_size = tmp.size()
    trailing_dims = max_size[1:]
    
    if batch_first:
        out_dims = (len(sequences), max_len) + trailing_dims
    else:
        out_dims = (max_len, len(sequences)) + trailing_dims

    out_tensor = tmp.data.new(*out_dims).fill_(padding_value)
    for i, list in enumerate(sequences):
        tensor = torch.Tensor(list)
        length = tensor.size(0)
        if batch_first:
            out_tensor[i, :length, ...] = tensor
        else:
            out_tensor[:length, i, ...] = tensor

    return out_tensor.long().numpy()

# def set_global_label(bio: bool = False) -> None:
#     global hash_token
#     global end_token 
#     if bio:
#         hash_token = 3
#         end_token = 4
#     else:
#         hash_token = 2
#         end_token = 3

def reg_encoding(cleaned: list, labels: list, hash_token, end_token) -> list:
    label_l = []
    for oindex, x in enumerate(cleaned):
        tlist = []
        for index, j in enumerate(x):
            for s in j:
                if s[0]=='#':
                    tlist.append(hash_token)
                else:
                    tlist.append(labels[oindex][index])
        label_l.append(tlist)
    return label_l

def bio_encoding(cleaned: list, labels: list) -> list:
    offset = 1
    
    label_l = []
    for oindex, x in enumerate(cleaned):
        tlist = []
        prev=labels[oindex][0]
        for index, j in enumerate(x):
            #if index==30:
            #ipdb.set_trace()
            for s in j:
                if s[0]=='#':
                    tlist.append(hash_token)
                else:
                    if (index==0 and labels[oindex][index]!=0):
                        tlist.append(labels[oindex][index]+offset)
                        prev = labels[oindex][index]
                    if (prev!=labels[oindex][index] and labels[oindex][index]!= 0):
                        tlist.append(labels[oindex][index]+offset)
                        prev = labels[oindex][index]
                    else:
                        tlist.append(labels[oindex][index])
                        prev = labels[oindex][index]
        label_l.append(tlist)
    return label_l

def concatenate_list_data(cleaned: list) -> list:
    result= []
    for element in cleaned:
        result += element
    return result

def make_set(p2id, data_dir: str, tokenizer, single_class: str, 
             hash_token, end_token, bio: bool = False) -> list: 
    #dataset = pd.read_csv(data_dir, sep='\t', header=None, converters={1:ast.literal_eval, 2:ast.literal_eval})
    data_dict = pickle.load(open(data_dir, "rb"))
    
    dataset = corpus2list(p2id, data_dict["ID"], data_dict["Text"],
                              data_dict["Label"], single_class, bio)
    # Shuffle samples
    #dataset = dataset.sample(frac=1)
    terms = list(dataset[1])
    labels = list(dataset[2])
    
    cleaned = [[tokenizer.tokenize(words) for words in sent] for sent in terms]
    tokenized_texts = [concatenate_list_data(sent) for sent in cleaned]
    if bio:
        label_l = bio_encoding(cleaned, labels)
    else:
        label_l = reg_encoding(cleaned, labels, hash_token, end_token)

    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          padding_value=0.0, max_len=opt["maxLen"])
    
    tags = pad_sequences(label_l, padding_value=end_token, max_len=opt["maxLen"])
    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
    
    
    return input_ids, tags, attention_masks, label_l


def make_val_set(p2id, data_dir: str, tokenizer, single_class: str, 
             hash_token, end_token, bio: bool = False) -> list: 
    #dataset = pd.read_csv(data_dir, sep='\t', header=None, converters={1:ast.literal_eval, 2:ast.literal_eval})
    data_dict = pickle.load(open(data_dir, "rb"))
    if not bio:
        dataset = corpus2list(p2id, data_dict["ID"], data_dict["Text"],
                              data_dict["Label"], single_class, bio)
    # Shuffle samples
    #dataset = dataset.sample(frac=1)
    ids = (dataset[0])
    terms = (dataset[1])
    labels = (dataset[2])
    spacy = (dataset[3])
    cleaned = [[tokenizer.tokenize(words) for words in sent] for sent in terms]
    tokenized_texts = [concatenate_list_data(sent) for sent in cleaned]

    if bio:
        label_l = bio_encoding(cleaned, labels)
    else:
        label_l = reg_encoding(cleaned, labels, hash_token, end_token)

    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          padding_value=0.0, max_len=opt["maxLen"])
    
    tags = pad_sequences(label_l, padding_value=end_token, max_len=opt["maxLen"])
    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
    
    
    return input_ids, tags, attention_masks, cleaned, ids, terms, spacy, label_l


In [13]:
opt = {
    "techniques" : "tools/data/propaganda-techniques-names.txt",
    "binaryLabel" : False,
    "nLabels" : 21,
    "lowerCase" : False,
    "bio" : False,
    "model" : "bert-base-cased",
    "trainDataset" : "train.p",
    "evalDataset" : "test.p",
    "trainBatch" :16,
    "nEpochs" :5,
    "LR" :3e-5,
    "loadModel" : "exp/all_class/all_class_proper/4/model_4.pth",
    "expID" :"spacy_viz",
    "classType" : "all_class",
    "testData" : "datasets-v5",
    "train" : True,
    "maxLen" : 210,
    "patience" : 2,
    "testDataset" : "datasets-v5/tasks-2-3/test",
    "snapshot" : 2
}

In [9]:
ls exp/all_class/all_class_proper/4

model_4.pth  optimizer.pth  option.pth  pred.tasks-2-3  score.tasks-2-3


In [5]:
def get_task2(predictions):
    preddi = []
    found = False
    for x in predictions:
        for j in x:
            if j==1:
                preddi.append(1)
                found = True
                break
        if not found:
            preddi.append(0)
        found = False
    return preddi

In [31]:
#os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3,4'
prop_tech_e, prop_tech, hash_token, end_token, p2id = settings(opt["techniques"], opt["binaryLabel"], opt["bio"])
logging.info("Training for class %s" % (opt["binaryLabel"]))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count(); 
logging.info("GPUs Detected: %s" % (n_gpu))
scorred_labels = list(range(1,(opt["nLabels"]-2)))

tokenizer = BertTokenizer.from_pretrained(opt["model"], do_lower_case=opt["lowerCase"]);
print (hash_token, end_token)
# Load Tokenized train and validation datasets
tr_inputs, tr_tags, tr_masks, _ = make_set(p2id, opt["trainDataset"], tokenizer, opt["binaryLabel"], hash_token, end_token)
val_inputs, val_tags, val_masks, cleaned, flat_list_i, flat_list, flat_list_s,_ = make_val_set(p2id, opt["evalDataset"],
                                                                                         tokenizer, opt["binaryLabel"], hash_token, end_token)
printable = tr_tags
# ids, texts, _ = read_data(opt["testDataset"], isLabels = False)
# flat_list_i, flat_list, flat_list_s = test2list(ids, texts)
truth_task2 = get_task2(val_tags)

logging.info("Dataset loaded")
logging.info("Labels detected in train dataset: %s" % (np.unique(tr_tags)))
logging.info("Labels detected in val dataset: %s" % (np.unique(val_tags)))

# Balanced Sampling
total_tags = np.zeros((opt["nLabels"],))
for x in tr_tags:
     total_tags = total_tags+np.bincount(x)

probs = 1./total_tags
train_tokenweights = probs[tr_tags]
weightage = np.sum(train_tokenweights, axis=1)
   # Alternate method for weighting
ws = np.ones((opt["nLabels"],))
ws[0] = 0

ws[hash_token] = 0
ws[end_token] = 0
ws = ws+0.3
prob = [max(x) for x in ws[tr_tags]]
weightage = [x + y for x, y in zip(prob, (len(prob)*[0.1]))]    

# Convert to pyTorch tensors
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

# Create Dataloaders
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
#train_sampler = WeightedRandomSampler(weights=weightage, num_samples=len(tr_tags),replacement=True)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=opt["trainBatch"])

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=opt["trainBatch"])

# Model Initialize
model = BertForTokenClassification.from_pretrained(opt["model"], num_labels=opt["nLabels"]);

loss_scale = 0
warmup_proportion = 0.1
num_train_optimization_steps = int(len(train_data) / opt["trainBatch"] ) * opt["nEpochs"]

# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not usedpython train.py --expID test --trainDataset dataset_train.csv --evalDataset dataset_dev.csv --model bert-base-cased --LR 3e-5 --trainBatch 12 --nEpochs 1
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# t_total matters
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=opt["LR"],
                     warmup=warmup_proportion,
                     t_total=num_train_optimization_steps) 

model.to(device)

if n_gpu > 1:
    model = torch.nn.DataParallel(model)
    logging.info("Training beginning on: %s" % n_gpu)

if opt["loadModel"]:
    print('Loading Model from {}'.format(opt["loadModel"]))
    model.load_state_dict(torch.load(opt["loadModel"]))
    if not os.path.exists("./exp/{}/{}".format(opt["classType"], opt["expID"])):
        try:
            os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))
        except FileNotFoundError:
            os.mkdir("./exp/{}".format(opt["classType"]))
            os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))
else:
    print('Create new model')
    if not os.path.exists("./exp/{}/{}".format(opt["classType"], opt["expID"])):
        try:
            os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))
        except FileNotFoundError:
            os.mkdir("./exp/{}".format(opt["classType"]))
            os.mkdir("./exp/{}/{}".format(opt["classType"], opt["expID"]))

# F1 score shouldn't consider no-propaganda
# and other auxiliary labels

global_step = 0
nb_tr_steps = 0
tr_loss = 0
max_grad_norm = 1.0
best = 0
early_stopping = EarlyStopping(patience=opt["patience"], verbose=True)
train_losses = []
valid_losses = []
f1_scores = []
f1_scores_word = []
task2_scores = []
for i in trange(opt["nEpochs"], desc="Epoch"):
    # TRAIN loop
    # Start only if train flag was passed
    if (opt["train"]):
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            if n_gpu == 1:
                batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            # forward pass
            loss = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
            if n_gpu > 1:
                loss = loss.mean()

            # backward pass
            loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        logging.info(f'EPOCH {i} done: Train Loss {(tr_loss/nb_tr_steps)}')
        train_losses.append(tr_loss/nb_tr_steps)

    # Evaluation on validation set or test set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in tqdm(valid_dataloader, desc="Evaluating"):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)

        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_loss += tmp_eval_loss.mean().item()
        #eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    pred_task2 = get_task2(predictions)
    logging.info("Precision, Recall, F1-Score, Support Task2: {}".format(f1(pred_task2, truth_task2, average=None)))
    f1_macro = f1_score(pred_task2, truth_task2, labels=scorred_labels, average="macro")
    task2_scores.append(f1_macro)
    pickle.dump(printable, open( "output_.p", "wb"))
    eval_loss = eval_loss/nb_eval_steps
    logging.info("Validation loss: %s" % (eval_loss))    
    logging.info("Precision, Recall, F1-Score, Support: {}".format(f1(list(itertools.chain(*predictions)), list(itertools.chain(*val_tags)), average=None)))
    f1_macro = f1_score(list(itertools.chain(*predictions)), list(itertools.chain(*val_tags)), labels=scorred_labels, average="macro")
    logging.info("F1 Macro Dev Set: %s" % f1_macro)
    logging.info("Learning Rate: %s" % (optimizer.get_lr()[0]))
    valid_losses.append(eval_loss)
    f1_scores_word.append(f1_macro)

    df = get_char_level(flat_list_i, flat_list_s, predictions, cleaned, hash_token, end_token, prop_tech)
    postfix = opt["testDataset"].rsplit('/', 2)[-2]
    if opt["loadModel"]:
        out_dir = opt["loadModel"].rsplit('/', 1)[0] + "/pred." + postfix
    else:
        out_dir = ("exp/{}/{}/temp_pred.csv".format(opt["classType"], opt["expID"]))
    df.to_csv(out_dir, sep='\t', index=False, header=False) 
    logging.info("Predictions written to: %s" % (out_dir))

    if opt["loadModel"]:
        out_file = opt["loadModel"].rsplit('/', 1)[0] + "/score." + postfix
    else:
        out_file = ("exp/{}/{}/temp_score.csv".format(opt["classType"], opt["expID"]))

    if opt["classType"] != "binary":
        char_predict = tools.task3_scorer_onefile.main(["-s", out_dir, "-r", opt["testDataset"], "-t", opt["techniques"], "-l", out_file])
    else:
        char_predict = tools.task3_scorer_onefile.main(["-s", out_dir, "-r", opt["testDataset"], "-t", opt["techniques"], "-f", "-l", out_file])
    f1_scores.append(char_predict) 
    print (char_predict)

    # early_stopping needs the validation loss to check if it has decresed, 
    # and if it has, it will make a checkpoint of the current model
    if not opt["train"]:
        break
    early_stopping(char_predict*(-1), model)

    if early_stopping.early_stop:
        logging.info("Early stopping")
        break
    # Save checkpoints
    if i % opt["snapshot"] == 0:
        if not os.path.exists("./exp/{}/{}/{}".format(opt["classType"], opt["expID"], i)):
            try:
                os.mkdir("./exp/{}/{}/{}".format(opt["classType"], opt["expID"], i))
            except FileNotFoundError:
                os.mkdir("./exp/{}/{}/{}".format(opt["classType"], opt["expID"], i))
        torch.save(
            model.state_dict(), './exp/{}/{}/{}/model_{}.pth'.format(opt["classType"], opt["expID"], i, i))
        torch.save(
            opt, './exp/{}/{}/{}/option.pth'.format(opt["classType"], opt["expID"], i))
        torch.save(
            optimizer, './exp/{}/{}/{}/optimizer.pth'.format(opt["classType"], opt["expID"], i))


    # Save model based on best F1 score and if epoch is greater than 3
    '''if f1_macro > best and i > 3:
    # Save a trained model and the associated configuration
        torch.save(
            model.state_dict(), './exp/{}/{}/best_model.pth'.format(opt["classType"], opt["expID"]))
        torch.save(
            opt, './exp/{}/{}/option.pth'.format(opt["classType"], opt["expID"]))
        torch.save(
            optimizer, './exp/{}/{}/optimizer.pth'.format(opt["classType"], opt["expID"]))
        #model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        #output_model_file = os.path.join("./exp/{}/{}".format(opt["classType"], opt["expID"]), "best_model.pth")
        #torch.save(model_to_save.state_dict(), output_model_file)
        best = f1_macro
        logging.info("New best model")
    '''
if opt["train"]:
    logging.info("Training Finished. Learning curves saved.")
    draw_curves(train_losses, valid_losses, f1_scores, f1_scores_word, task2_scores)
    #df = pd.DataFrame({'col':trainlosses})
    #df.to_csv("trainlosses.csv", sep='\t', index=False, header=False) 
    #df = pd.DataFrame({'col':validlosses})
    #df.to_csv("validlosses.csv", sep='\t', index=False, header=False) 
    #df = pd.DataFrame({'col':f1scores})
    #df.to_csv("f1scores.csv", sep='\t', index=False, header=False) 



19 20
{'O': 0, 'Appeal_to_Authority': 1, 'Appeal_to_fear-prejudice': 2, 'Bandwagon': 3, 'Black-and-White_Fallacy': 4, 'Causal_Oversimplification': 5, 'Doubt': 6, 'Exaggeration,Minimisation': 7, 'Flag-Waving': 8, 'Loaded_Language': 9, 'Name_Calling,Labeling': 10, 'Obfuscation,Intentional_Vagueness,Confusion': 11, 'Red_Herring': 12, 'Reductio_ad_hitlerum': 13, 'Repetition': 14, 'Slogans': 15, 'Straw_Men': 16, 'Thought-terminating_Cliches': 17, 'Whataboutism': 18}
{'O': 0, 'Appeal_to_Authority': 1, 'Appeal_to_fear-prejudice': 2, 'Bandwagon': 3, 'Black-and-White_Fallacy': 4, 'Causal_Oversimplification': 5, 'Doubt': 6, 'Exaggeration,Minimisation': 7, 'Flag-Waving': 8, 'Loaded_Language': 9, 'Name_Calling,Labeling': 10, 'Obfuscation,Intentional_Vagueness,Confusion': 11, 'Red_Herring': 12, 'Reductio_ad_hitlerum': 13, 'Repetition': 14, 'Slogans': 15, 'Straw_Men': 16, 'Thought-terminating_Cliches': 17, 'Whataboutism': 18}







Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Create new model


0
Counter check:  0
{'submission': 'exp/all_class/spacy_viz/temp_pred.csv', 'gold': 'datasets-v5/tasks-2-3/test', 'debug_on_std': False, 'techniques_file': 'tools/data/propaganda-techniques-names.txt', 'log_file': 'exp/all_class/spacy_viz/temp_score.csv', 'fragments_only': False}
2019-06-19 08:50:46,250 - INFO - Logging execution to file exp/all_class/spacy_viz/temp_score.csv


INFO:propaganda_scorer:Logging execution to file exp/all_class/spacy_viz/temp_score.csv


...

2019-06-19 09:25:59,978 - INFO - Whataboutism: P=0.000000 R=0.000000 F1=0.000000


INFO:propaganda_scorer:Whataboutism: P=0.000000 R=0.000000 F1=0.000000


0.1447982455156738
Validation loss decreased (-0.137587 --> -0.144798).  Saving model ...







Epoch: 100%|██████████| 5/5 [44:04<00:00, 529.74s/it]

NameError: name 'draw_curves' is not defined

In [15]:
 os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3,4'

In [16]:
MAX_LEN = 210
bs = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count(); 
logging.info("GPUs Detected: %s" % (n_gpu))

tokenizer = BertTokenizer.from_pretrained(opt["model"], do_lower_case=False);
# Model Initialize
model = BertForTokenClassification.from_pretrained(opt["model"], num_labels=opt["nLabels"]);

model.to(device)

model.load_state_dict(torch.load(opt["loadModel"]))

RuntimeError: Error(s) in loading state_dict for BertForTokenClassification:
	Missing key(s) in state_dict: "bert.embeddings.word_embeddings.weight", "bert.embeddings.position_embeddings.weight", "bert.embeddings.token_type_embeddings.weight", "bert.embeddings.LayerNorm.weight", "bert.embeddings.LayerNorm.bias", "bert.encoder.layer.0.attention.self.query.weight", "bert.encoder.layer.0.attention.self.query.bias", "bert.encoder.layer.0.attention.self.key.weight", "bert.encoder.layer.0.attention.self.key.bias", "bert.encoder.layer.0.attention.self.value.weight", "bert.encoder.layer.0.attention.self.value.bias", "bert.encoder.layer.0.attention.output.dense.weight", "bert.encoder.layer.0.attention.output.dense.bias", "bert.encoder.layer.0.attention.output.LayerNorm.weight", "bert.encoder.layer.0.attention.output.LayerNorm.bias", "bert.encoder.layer.0.intermediate.dense.weight", "bert.encoder.layer.0.intermediate.dense.bias", "bert.encoder.layer.0.output.dense.weight", "bert.encoder.layer.0.


# Evaluation

## Test

In [ ]:
np.set_printoptions(threshold=1000)

In [32]:
def visualize(index, predictions, tokenized_texts, test = True, val_tags=None):
    print('Sentence: '+' '.join(tokenized_texts[index]))
    pred_l = [predictions[index][i] for i in range(len(predictions[index])) if predictions[index][i] not in [hash_token, end_token]]
    pred_l = max(pred_l)
    if not test:
        truth = [i for i in range(len(val_tags[index])) if val_tags[index][i] not in [hash_token, end_token, 0]]
        truth_l = [val_tags[index][i] for i in range(len(val_tags[index])) if val_tags[index][i] not in [hash_token, end_token]]
        truth_l = max(truth_l)
        gold = [tokenized_texts[index][i] for i in truth]
        print('Truth label: ',truth_l, ' Predicted label: ',pred_l)
        print('Porpaganda Sequence: '+' '.join(gold))
 
    
    pred = [i for i in range(len(predictions[index])) if predictions[index][i] not in [hash_token, end_token, 0]]
    
    if pred:
        if test:
            print(len(predictions))
            print(predictions)
            #print([tag_t.get(predictions[0][x]) for x in predictions[0]])
        predicted = [tokenized_texts[index][i] for i in pred]
        print('Predicted Sequence: '+' '.join(predicted))
        #print(predictions)
    else:
        print("No propaganda detected")

In [33]:
visualize(4, predictions, tokenized_texts, False, val_tags.detach().cpu().numpy().tolist())

NameError: name 'tokenized_texts' is not defined

In [42]:
def test_sentence(sample):
    sample = sample.split()
    clean = [[tokenizer.tokenize(words) for words in sent] for sent in [sample]] 
    tokenize = [concatenate_list_data(sent) for sent in clean]
    numerics = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenize],
                          max_len=opt["maxLen"])
    attention_masks = [[float(i>0) for i in ii] for ii in numerics]
    input_sample = torch.tensor(numerics)
    input_mask = torch.tensor(attention_masks)
    model.eval()
    logits = model(input_sample, token_type_ids=None,
                       attention_mask=input_mask)
    predictions_sample = []
    logits = logits.detach().cpu().numpy()
    predictions_sample.extend([list(p) for p in np.argmax(logits, axis=2)])
    visualize(0, predictions_sample, tokenize)

In [47]:
test_sentence('Trump is a piece of trash.')

Sentence: Trump is a piece of trash .
1
[[0, 0, 0, 9, 9, 9, 0, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]]
Predicted Sequence: piece of trash


In [134]:
x = 555

In [135]:
viz = df.loc[df['ID'] == df["ID"][x]]

In [136]:
strPath = "datasets-v5/tasks-2-3/test/article"+df["ID"][x]+".txt"

f = open(strPath)

strText = f.read()

docs = {"text":strText,
       "ents":[],
       "title":None}

for index, row in viz.iterrows():
    docs["ents"].append({"start":row[2], "end":row[3], "label":row[1]})

In [137]:
from spacy import displacy
displacy.render(docs, style="ent", manual=True)